# Álgebra Linear Computacional - CKP8122 - MDCC - UFC
### Francisco Mateus dos Anjos Silva
# SOR

Em álgebra linear numérica, o **método de superrelaxação sucessiva (SOR)** é uma variante do método de Gauss-Seidel para resolver um sistema linear de equações, resultando em uma convergência mais rápida. Um método semelhante pode ser usado para qualquer processo iterativo de convergência lenta.

Foi concebido simultaneamente por David M. Young Jr. e por Stanley P. Frankel em 1950 com o propósito de resolver automaticamente sistemas lineares em computadores digitais. Métodos de relaxamento excessivo foram usados antes do trabalho de Young e Frankel. Um exemplo é o método de Lewis Fry Richardson, e os métodos desenvolvidos por RV Southwell. No entanto, esses métodos foram projetados para computação por calculadoras humanas, exigindo algum conhecimento para garantir a convergência para a solução, o que os tornava inaplicáveis para programação em computadores digitais.

Dado um sistema quadrado de n equações lineares com x desconhecido:

${\displaystyle A\mathbf {x} =\mathbf {b} }$

Onde:

${\displaystyle A={\begin{bmatrix}a_{11}&a_{12}&\cdots &a_{1n}\\a_{21}&a_{22}&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &a_{nn}\end{bmatrix}},\qquad \mathbf {x} ={\begin{bmatrix}x_{1}\\ x_{2}\\\vdots \\x_{n}\end{bmatrix}},\qquad \mathbf {b} ={\begin{bmatrix}b_{1}\\b_{2}\\\vdots \\b_{n}\end{bmatrix}}.}$

Então A pode ser decomposto em um componente diagonal D, e componentes triangulares estritamente inferiores e superiores L e U:

${A=D+L+U,}$

Onde

${\displaystyle D={\begin{bmatrix}a_{11}&0&\cdots &0\\0&a_{22}&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &a_{nn }\end{bmatrix}},\quad L={\begin{bmatrix}0&0&\cdots &0\\a_{21}&0&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\a_{n1 }&a_{n2}&\cdots &0\end{bmatrix}},\quad U={\begin{bmatrix}0&a_{12}&\cdots &a_{1n}\\0&0&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &0\end{bmatrix}}.}$

O sistema de equações lineares pode ser reescrito como:

${\displaystyle (D+\omega L)\mathbf {x} =\omega \mathbf {b} -[\omega U+(\omega -1)D]\mathbf {x} }$

para uma constante ω > 1, chamada de fator de relaxação.

O método SOR é uma técnica iterativa que resolve o lado esquerdo desta expressão para x, usando o valor anterior para x no lado direito. Analiticamente, isso pode ser escrito como:

${\displaystyle \mathbf {x} ^{(k+1)}=(D+\omega L)^{-1}{\big (}\omega \mathbf {b} -[\omega U+(\omega -1 )D]\mathbf {x} ^{(k)}{\big )}=L_{w}\mathbf {x} ^{(k)}+\mathbf {c} ,}$ onde ${\displaystyle \mathbf {x} ^{(k)}}$ é a k-ésima aproximação ou iteração de ${\displaystyle \mathbf {x} }$ e ${ \displaystyle \mathbf {x} ^{(k+1)}}$ é a próxima ou k + 1 iteração de ${\displaystyle \mathbf {x} }$. No entanto, aproveitando a forma triangular de (D+ωL), os elementos de x(k+1) podem ser calculados sequencialmente usando a substituição direta:

${\displaystyle x_{i}^{(k+1)}=(1-\omega )x_{i}^{(k)}+{\frac {\omega }{a_{ii}}}\left( b_{i}-\sum _{j<i}a_{ij}x_{j}^{(k+1)}-\sum _{j>i}a_{ij}x_{j}^{(k )}\right),\quad i=1,2,\ldots ,n.}$

**Referências:**

- https://en.wikipedia.org/wiki/Successive_over-relaxation

In [1]:
# Implemente o método SOR (Successive OverRelaxation) para resolver, de forma iterativa, um sistema de equações 
# algébricas lineares.

# Teste vários valores do parâmetro ômega para ver sua influência na aceleração da convergência.

In [2]:
import numpy as np

In [3]:
def SOR(A, b, omega=0.5, error=0.00001, max_iter=500):
    n_A = len(A)
    solutions = np.zeros(n_A)
    previous_solutions =np.zeros(n_A)
    qtd_iter = 1

    while qtd_iter <= max_iter:
        solutions = np.zeros(n_A)
        
        for i in range(0, n_A):
            sum_others = 0
            for j in range(0, n_A):
                if i != j:
                    if j < i:
                        sum_others += A[i,j] * solutions[j]
                    else:
                        sum_others += A[i,j] * previous_solutions[j]
            solutions[i] = (1-omega) * previous_solutions[i] + (omega/A[i,i]) * (b[i] - sum_others)
            
        if np.allclose(previous_solutions, solutions, atol=error, rtol=0.):
            break
        
        qtd_iter +=1
        previous_solutions = solutions;
        
    print('\nQuantidade de iterações:', qtd_iter)
    return solutions

In [4]:
# Tranformar diagonal de A em diagonal dominante
def diag_dominant(A):
    for i in range(0, len(A)):
        A[i,i] = (np.sum(abs(A[i])) - abs(A[i,i])) + 1
    return A

In [5]:
np.set_printoptions(precision=4)

A = np.array([[ 3,-2, 1, 1],
              [-2, 3, 0, 2],
              [ 1, 0, 2, 0],
              [ 1, 2, 0, 3]])

A = diag_dominant(A)

b = np.array([[4],
              [3], 
              [2], 
              [3]])

print('Matriz A:\n', A)
print('\nb:\n', b)

print('\nSolution (numpy):', np.linalg.solve(A,b).flatten())

for omega in np.arange(0.1,1,0.1):
    solutions_sor = SOR(A, b, omega=omega)
    print('Solução SOR com omega =',omega,':',solutions_sor)

Matriz A:
 [[ 5 -2  1  1]
 [-2  5  0  2]
 [ 1  0  2  0]
 [ 1  2  0  4]]

b:
 [[4]
 [3]
 [2]
 [3]]

Solution (numpy): [ 1.186   1.1163  0.407  -0.1047]

Quantidade de iterações: 303
Solução SOR com omega = 0.1 : [ 1.1857  1.1159  0.4072 -0.1043]

Quantidade de iterações: 158
Solução SOR com omega = 0.2 : [ 1.1859  1.1161  0.4071 -0.1045]

Quantidade de iterações: 105
Solução SOR com omega = 0.30000000000000004 : [ 1.1859  1.1162  0.407  -0.1046]

Quantidade de iterações: 76
Solução SOR com omega = 0.4 : [ 1.186   1.1162  0.407  -0.1046]

Quantidade de iterações: 59
Solução SOR com omega = 0.5 : [ 1.186   1.1162  0.407  -0.1046]

Quantidade de iterações: 47
Solução SOR com omega = 0.6 : [ 1.186   1.1162  0.407  -0.1046]

Quantidade de iterações: 38
Solução SOR com omega = 0.7000000000000001 : [ 1.186   1.1163  0.407  -0.1046]

Quantidade de iterações: 31
Solução SOR com omega = 0.8 : [ 1.186   1.1163  0.407  -0.1046]

Quantidade de iterações: 25
Solução SOR com omega = 0.9 : [ 1.186   1.

Podemoss observar que quanto maior o omega, menor a quantidade de iterações e o resultado é mais preciso.